<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "32g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')
DATA_PROCESSING_MODEL_CHECKPOINT = os.path.join(DATA_PATH, 'data_processing_model.bin')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [4]:
df = df.sample(False, 0.1)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [7]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(DATA_PROCESSING_MODEL_CHECKPOINT)

In [8]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

366157

In [9]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [11]:
!pip3.5 install hyperopt

     |████████████████████████████████| 1.9 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 21.0 MB/s eta 0:00:01
    Running setup.py install for networkx ... done
    Running setup.py install for future ... done


## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [10]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [11]:
static_params = {
    'featuresCol': 'features', 
    'labelCol': 'label', 
    'predictionCol': 'prediction',
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [12]:
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [13]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.7212021011149682

In [14]:
all_metrics = {}

In [15]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [17]:
%%time

num_round_choice = [50, 100]
eta_choice = [0.1, 0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials
)

SPACE:                                                
{'max_depth': 6, 'predictionCol': 'prediction', 'subsample': 0.9, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.9, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'eta': 0.1}
LOG-LOSS: 0.508318383028313, ROC-AUC: 0.727585969611326
SPACE:                                                                         
{'max_depth': 6, 'predictionCol': 'prediction', 'subsample': 0.9, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.9, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'eta': 0.5}
LOG-LOSS: 0.5087452285049104, ROC-AUC: 0.7272515883610267                      
SPACE:                                                                         
{'max_depth': 6, 'predictionCol

LOG-LOSS: 0.5074302948033367, ROC-AUC: 0.7288860675846821                        
SPACE:                                                                           
{'max_depth': 6, 'predictionCol': 'prediction', 'subsample': 0.9, 'eval_metric': 'logloss', 'num_round': 50, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.9, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'eta': 0.1}
LOG-LOSS: 0.5104478670949751, ROC-AUC: 0.7246221335251408                        
SPACE:                                                                           
{'max_depth': 6, 'predictionCol': 'prediction', 'subsample': 0.9, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.9, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'eta': 0.5}
LOG-LOSS: 0.5087452285049104, ROC-AUC: 0.72725158836102

In [18]:
best

{'eta': 5, 'num_round': 1}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [19]:
eta = eta_choice[best['eta']]  # change me!
num_round = num_round_choice[best['num_round']]  # change me!

In [20]:
eta, num_round

(0.3, 100)

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

In [21]:
max_depth_choice = [4, 6, 8]
min_child_weight_choice = [1.0, 1.5, 2.0]
subsample_choice = [0.7, 0.8, 0.9, 1.0]
colsample_bytree_choice = [0.7, 0.8, 0.9, 1.0]
gamma_choice = [0., 0.2, 0.4, 0.9]

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    'gamma': hp.choice('gamma', gamma_choice),
    
    # Fixed    
    'num_round': num_round,
    'eta': eta,
    
    **static_params
}

In [22]:
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials
)

SPACE:                                                
{'max_depth': 6, 'predictionCol': 'prediction', 'subsample': 1.0, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.0, 'min_child_weight': 1.0, 'colsample_bytree': 0.9, 'eta': 0.3}
LOG-LOSS: 0.5080751015654534, ROC-AUC: 0.7273941031163166
SPACE:                                                                          
{'max_depth': 6, 'predictionCol': 'prediction', 'subsample': 0.8, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.2, 'min_child_weight': 2.0, 'colsample_bytree': 0.9, 'eta': 0.3}
LOG-LOSS: 0.5062359215479508, ROC-AUC: 0.7309382392201618                       
SPACE:                                                                          
{'max_depth': 8, 'prediction

LOG-LOSS: 0.5089029444724141, ROC-AUC: 0.726677086951242                         
SPACE:                                                                           
{'max_depth': 6, 'predictionCol': 'prediction', 'subsample': 0.9, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.9, 'min_child_weight': 1.0, 'colsample_bytree': 0.7, 'eta': 0.3}
LOG-LOSS: 0.5064457483378872, ROC-AUC: 0.7301618595661555                        
SPACE:                                                                           
{'max_depth': 8, 'predictionCol': 'prediction', 'subsample': 0.9, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.4, 'min_child_weight': 2.0, 'colsample_bytree': 0.8, 'eta': 0.3}
LOG-LOSS: 0.5082453371659044, ROC-AUC: 0.727285430269515

In [23]:
best

{'colsample_bytree': 2,
 'gamma': 1,
 'max_depth': 1,
 'min_child_weight': 2,
 'subsample': 1}

In [24]:
max_depth = max_depth_choice[best['max_depth']]
min_child_weight = min_child_weight_choice[best['min_child_weight']]
subsample = subsample_choice[best['subsample']]
colsample_bytree = colsample_bytree_choice[best['colsample_bytree']]
gamma = gamma_choice[best['gamma']]

In [25]:
max_depth, min_child_weight, subsample, colsample_bytree, gamma

(6, 2.0, 0.8, 0.9, 0.2)

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [26]:
alpha_choice = [0.0, 0.1, 0.4, 0.9, 1.6, 2.5]
lambda_choice = [0.0, 0.1, 0.4, 0.9, 1.6, 2.5]

space = {
    # Optimize
    'alpha': hp.choice('alpha', alpha_choice),
    'reg_lambda': hp.choice('lambda', lambda_choice),
    
    # Fixed    
    'num_round': num_round,
    'eta': eta,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'gamma': gamma,
    
    **static_params
}

In [27]:
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials
)

SPACE:                                                
{'max_depth': 6, 'predictionCol': 'prediction', 'alpha': 0.0, 'subsample': 0.8, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.2, 'min_child_weight': 2.0, 'lambda': 2.5, 'colsample_bytree': 0.9, 'eta': 0.3}
LOG-LOSS: 0.5061853844614433, ROC-AUC: 0.7306568487694753
SPACE:                                                                          
{'max_depth': 6, 'predictionCol': 'prediction', 'alpha': 1.6, 'subsample': 0.8, 'eval_metric': 'logloss', 'num_round': 100, 'nworkers': 1, 'nthread': 1, 'labelCol': 'label', 'featuresCol': 'features', 'silent': 0, 'objective': 'binary:logistic', 'gamma': 0.2, 'min_child_weight': 2.0, 'lambda': 0.0, 'colsample_bytree': 0.9, 'eta': 0.3}
LOG-LOSS: 0.5068895761667104, ROC-AUC: 0.7297421174138728                       
SPACE:                                             

In [28]:
best

{'alpha': 4, 'lambda': 4}

In [29]:
alpha = alpha_choice[best['alpha']]
reg_lambda = lambda_choice[best['lambda']]

In [30]:
alpha, reg_lambda

(1.6, 1.6)

### Lower the learning rate and decide the optimal parameters

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

### Validation

In [31]:
xgb_tuned_params = {
    'num_round': num_round,
    'eta': eta,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'gamma': gamma,
    'alpha': alpha,
    'reg_lambda': reg_lambda,
    **static_params
}

xgb_tuned_model = XGBoostEstimator(**xgb_tuned_params).fit(train_df)

In [57]:
xgb_tuned_params

{'alpha': 1.6,
 'colsample_bytree': 0.9,
 'eta': 0.3,
 'eval_metric': 'logloss',
 'featuresCol': 'features',
 'gamma': 0.2,
 'labelCol': 'label',
 'max_depth': 6,
 'min_child_weight': 2.0,
 'nthread': 1,
 'num_round': 100,
 'nworkers': 1,
 'objective': 'binary:logistic',
 'predictionCol': 'prediction',
 'reg_lambda': 1.6,
 'silent': 0,
 'subsample': 0.8}

In [32]:
xgb_tuned_rocauc = rocauc(xgb_tuned_model, val_df, probabilities_col='probabilities')
xgb_tuned_rocauc

0.7309250700230058

In [33]:
xgb_tuned_test_metrics = {
    'logloss': logloss(xgb_tuned_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(xgb_tuned_model, test_df, probabilities_col='probabilities')
}

all_metrics['xgb tuned'] = xgb_tuned_test_metrics

In [34]:
get_ate(all_metrics, CONTROL_NAME)

,metric,xgb tuned ate %
0,logloss,-1.407397
1,rocauc,1.306159


---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [35]:
from pyspark.ml.classification import LogisticRegression

In [36]:
lr_baseline_params = {
    'maxIter': 10,
    'regParam': 0.0,
    'elasticNetParam': 0.0,
    'tol': 1e-06
}

In [37]:
lr_model = LogisticRegression(**lr_baseline_params).fit(train_df)

In [38]:
lr_baseline_rocauc = rocauc(lr_model, val_df, probabilities_col='probability')
lr_baseline_rocauc

0.6976546576855418

In [39]:
lr_test_metrics = {
    'logloss': logloss(lr_model, test_df, probabilities_col='probability'),
    'rocauc': rocauc(lr_model, test_df, probabilities_col='probability')
}

all_metrics['lr'] = lr_test_metrics

In [40]:
def objective(space):
    model = LogisticRegression(**space).fit(train_df)
    log_loss = logloss(model, val_df, probabilities_col='probability')
    roc_auc = rocauc(model, val_df, probabilities_col='probability')
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))
    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [41]:
max_iter_choice = [10, 20, 30]
reg_param_choice = [0.0, 0.1, 0.4, 0.9, 1.0]
elastic_net_param_choice = [0.0, 0.1, 0.4, 0.9, 1.0]

space = {
    'maxIter': hp.choice('max_iter', max_iter_choice),
    'regParam': hp.choice('reg_param', reg_param_choice),
    'elasticNetParam': hp.choice('elastic_net_param', elastic_net_param_choice)
}

In [42]:
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=30,
    trials=trials
)

LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5            
LOG-LOSS: 0.5675445996191201, ROC-AUC: 0.6594224016327341                       
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5588103527489803, ROC-AUC: 0.6942729172782734                       
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5357839771359361, ROC-AUC: 0.6973180552409294                        
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5758118339516891, ROC-AUC: 0.5          

In [43]:
best

{'elastic_net_param': 0, 'max_iter': 0, 'reg_param': 0}

In [44]:
max_iter = max_iter_choice[best['max_iter']]
reg_param = reg_param_choice[best['reg_param']]
elastic_net_param = elastic_net_param_choice[best['elastic_net_param']]

In [45]:
max_iter, reg_param, elastic_net_param

(10, 0.0, 0.0)

In [46]:
lr_tuned_params = {
    'maxIter': max_iter,
    'regParam': reg_param,
    'elasticNetParam': elastic_net_param
}

In [47]:
lr_tuned_model = LogisticRegression(**lr_tuned_params).fit(train_df)

In [48]:
lr_tuned_rocauc = rocauc(lr_tuned_model, val_df, probabilities_col='probability')
lr_tuned_rocauc

0.6976546576855414

In [49]:
lr_tuned_test_metrics = {
    'logloss': logloss(lr_tuned_model, test_df, probabilities_col='probability'),
    'rocauc': rocauc(lr_tuned_model, test_df, probabilities_col='probability')
}

all_metrics['lr tuned'] = lr_tuned_test_metrics

In [50]:
get_ate(all_metrics, CONTROL_NAME)

,lr ate %,lr tuned ate %,metric,xgb tuned ate %
0,4.259000,4.259000,logloss,-1.407397
1,-3.720112,-3.720112,rocauc,1.306159


---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

In [51]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

df = df.fillna(0, subset=num_columns)

df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [52]:
all_metrics = {}

In [53]:
baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)
all_metrics[CONTROL_NAME] = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

In [54]:
xgb_tuned_model = XGBoostEstimator(**xgb_tuned_params).fit(train_df)
all_metrics['xgb tuned'] = {
    'logloss': logloss(xgb_tuned_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(xgb_tuned_model, test_df, probabilities_col='probabilities')
}

In [55]:
lr_model = LogisticRegression(**lr_baseline_params).fit(train_df)
all_metrics['lr'] = {
    'logloss': logloss(lr_model, test_df, probabilities_col='probability'),
    'rocauc': rocauc(lr_model, test_df, probabilities_col='probability')
}

In [56]:
lr_tuned_model = LogisticRegression(**lr_tuned_params).fit(train_df)
all_metrics['lr tuned'] = {
    'logloss': logloss(lr_tuned_model, test_df, probabilities_col='probability'),
    'rocauc': rocauc(lr_tuned_model, test_df, probabilities_col='probability')
}

Итоговая таблица

In [58]:
get_ate(all_metrics, CONTROL_NAME)

,lr ate %,lr tuned ate %,metric,xgb tuned ate %
0,3.215517,3.215517,logloss,-1.899988
1,-3.188346,-3.188346,rocauc,1.722296


In [59]:
all_metrics

{'lr': {'logloss': 0.5302156399130152, 'rocauc': 0.7021411411153459},
 'lr tuned': {'logloss': 0.5302156399130152, 'rocauc': 0.7021411411153458},
 'xgb baseline': {'logloss': 0.5136976051971426, 'rocauc': 0.7252651031984296},
 'xgb tuned': {'logloss': 0.5039374117544929, 'rocauc': 0.7377563163538979}}